# 条件付き確率で考える

単純ベイズ推定器を使ったクラスタリングをベースに、各条件における勝ち・負け・引き分けになる確率を計算し、それぞれを掛け合わせて一番高い確率のものを採用することにする。

* [データ収集](001_データ収集.ipynb)で収集したデータを使う。
* [データ加工](002_データ加工.ipynb)で加工済みとする。

In [1]:
import download
import process_data
import pandas as pd
df = download.get_db_data()
df = process_data.processing_data(df)
#display(df)

# 符号化

* チーム名・スタジアム名を符号化
* 勝ち・負け・引き分けを符号化
* 試合日を月のみ取得

In [2]:
home = pd.concat([df["ホーム"],df["アウェイ"]]).drop_duplicates().reset_index(drop=True).to_dict()
team_dict = {v:k for k, v in home.items()} # 辞書のキー・バリュー交換

In [3]:
tmp = [team_dict[df["ホーム"][i]] for i in range(len(df))]
df["home"] = tmp
tmp = [team_dict[df["アウェイ"][i]] for i in range(len(df))]
df["away"] = tmp
#display(df)

In [4]:
studium = df["スタジアム"].drop_duplicates().reset_index(drop=True).to_dict()
studium_dict = {v:k for k, v in studium.items()}

In [5]:
tmp = [studium_dict[df["スタジアム"][i]] for i in range(len(df))]
df["studium"] = tmp

勝敗はホームチームの勝ち(WIN)、負け(LOSE)、引き分け(DRAW)の三種類に符号化する。  
最終出力を獲得勝ち点で出力することを考え、WIN=3/DRAW=1/LOSE=0としておく。

カップ戦とかでPK戦の結果も含まれているが、PKは引き分けととらえることにする。

In [6]:
import re
import numpy as np
wdl = []
# win-draw-lose VゴールとかPKとかはTOTO予想に関係ないので無視
LOSE = 0
DRAW = 1
WIN = 3
OTHER = np.nan
for result in df["スコア"]:
    tmp = re.split("[-()]",result)
    if len(tmp) < 2: # X-Xという形式でないものはスルー
        wdl.append(OTHER)
        continue
    if int(tmp[0]) > int(tmp[1]):
        wdl.append(WIN)
    elif int(tmp[0]) < int(tmp[1]):
        wdl.append(LOSE)
    else:
        wdl.append(DRAW)
df["result"] = wdl
#display(df)

対戦した時期として、月の情報を入れるものとする。

試合日から情報を検出する。

In [7]:
month = []
for match_day in df["試合日"]:
    tmp = match_day.split("/")[0]
    if tmp.isdigit() == True:
        month.append(int(tmp))
    else:
        month.append(np.nan)
df["month"] = month
#display(month)

In [8]:
df = df.rename(columns={"年度":"year"})
#display(df)

使うデータを1992年~2016年に絞る。  
2017年のデータは予測の正確性検証のために別にしておくため

In [9]:
data = df[df["year"] < 2017]

結果の異常値を弾く。

異常値としてNaN(np.nan)を使ったので、NaNを含む列をドロップする。

In [10]:
data = data.dropna()

ホームチームを限定して、入力データと2017年のデータを使った正解データを生成してみる。

In [11]:
def generate_trainng_data(df, year=2017, home=None, away=None):
    """
    ホームチームを指定し、学習用データ(指定した年度より前のデータから)を生成
    """
    data = df[df["year"] < year].dropna()
    if home is not None and away is not None:
        data = data[((data["ホーム"]==home) | (data["アウェイ"]==home))|((data["ホーム"]==away) | (data["アウェイ"]==away))]
    x = data[["home","away","year","month","studium"]]
    y = data["result"]
    return x, y

def generate_correct_data(df, year=2017, home=None, away=None):
    """
    ホームチームを指定し、モデルの確からしさを検証するためのデータを生成
    """
    data = df[df["year"] == year].dropna()
    if home is not None and away is not None:
        data = data[((data["ホーム"]==home) | (data["アウェイ"]==home))|((data["ホーム"]==away) | (data["アウェイ"]==away))]
    x = data[["home","away","year","month","studium"]]
    y = data[["result"]]
    return x, y

## 検討する条件

1. チームの通算成績における勝率(ベースの分布？)  
$\Rightarrow \dfrac{勝利した数}{通算試合数}$
2. 対戦相手を限定した場合の勝率  
$\Rightarrow \dfrac{対戦相手に勝利した数}{対戦相手との通算試合数}$
3. 試合開催が何月かで限定した場合の勝率  
$\Rightarrow \dfrac{〇月の試合の中で勝利した数}{〇月に行われた試合数}$
4. 試合開催がどのスタジアムで限定した勝率  
$\Rightarrow \dfrac{指定スタジアムでの勝利数}{指定スタジアムでの試合数}$

In [11]:
def get_team_data(team, df):
    """
    指定したチームに関するデータのみ抽出する
    """
    team_data = df[(df["ホーム"]==team) | (df["アウェイ"]==team)]
    return team_data

def get_win_data(team, df, result=3):
    """
    指定したチームが勝利したデータのみ抽出する
    引数resultを指定すると、引き分け/負けのデータも収集可能
    """
    if result == 3:
        win_data = df[((df["ホーム"]==team) & (df["result"]==3)) | ((df["アウェイ"]==team) & (df["result"]==0))]
    elif result == 1:
        win_data = df[(df["result"]==1)]
    elif result == 0:
        win_data = df[((df["ホーム"]==team) & (df["result"]==0)) | ((df["アウェイ"]==team) & (df["result"]==3))]        
    return win_data

def get_studium_data(studium, df):
    """
    指定したスタジアムでのデータのみ抽出する
    """
    studium_data = df[(df["スタジアム"]==studium)]
    return studium_data

def get_month_data(month, df):
    """
    指定した月のデータのみ抽出する
    """
    month_data = df[df["month"]==month]
    return month_data

In [12]:
def proba_all(team, data, index="all"):
    """
    全試合に関する勝率を計算
    """
    team_data = get_team_data(team, data)
    if len(team_data)==0:
        return None
    win_data = get_win_data(team, team_data, result=3)
    draw_data = get_win_data(team, team_data, result=1)
    lose_data = get_win_data(team, team_data, result=0)
    ret = pd.DataFrame()
    ret["win"] = [len(win_data)/len(team_data)]
    ret["draw"] = [len(draw_data)/len(team_data)]
    ret["lose"] = [len(lose_data)/len(team_data)]
    #print(len(win_data),len(draw_data),len(lose_data),len(team_data))
    ret = ret.rename(index={0:index})
    return ret

def proba_oppotunity(team, oppotunity, data):
    """
    対戦相手を限定した際の勝率を計算
    """
    team_data = get_team_data(team, data)
    team_data = get_team_data(oppotunity, team_data)
    return proba_all(team, team_data,index="oppotunity")

def proba_month(team, month, data):
    """
    開催日が何月かで限定した際の勝率を計算
    """
    team_data = get_team_data(team, data)
    month_data = get_month_data(month, team_data)
    #display(month_data)
    return proba_all(team,month_data,index="month")

def proba_quarter(team, month, data):
    """
    開催月を四半期ごとに分けて勝率を計算
    Jリーグはだいたい2月終わり～12月初めまで。12月からは天皇杯。
    """
    QUARTERS = [[1,2,3],[4,5,6],[7,8,9],[10,11,12]]
    if month >= 1 or month <= 3:
        q = 0
    elif month >= 4 and month <= 6:
        q = 1
    elif month >= 7 and month <= 9:
        q = 2
    else:
        q = 3
    team_data = get_team_data(team, data)
    quarter_data = data[(data["month"]==QUARTERS[q][0]) | (data["month"]==QUARTERS[q][1]) | (data["month"]==QUARTERS[q][2])]
    #display(quarter_data)
    return proba_all(team,quarter_data,index="quarter")

def proba_studium(team, studium, data):
    """
    指定スタジアムでの試合に限定した勝率を計算
    """
    team_data = get_team_data(team, data)
    studium_data = get_studium_data(studium, team_data)
    return proba_all(team, studium_data,index="studium")

HOME = "川崎Ｆ"
AWAY = "札幌"
STUDIUM = "等々力"
tmp = pd.DataFrame()
tmp = tmp.append(proba_all(HOME, df))
tmp = tmp.append(proba_oppotunity(HOME, AWAY, df))
tmp = tmp.append(proba_month(HOME,9,df))
tmp = tmp.append(proba_studium(HOME, STUDIUM, df))
display(tmp.product().idxmax())

'win'

In [13]:
def prediction(home, away, month, studium, df):
    tmp = pd.DataFrame()
    proba = proba_all(home, df)
    if proba is not None:
        tmp = tmp.append(proba)
    proba = proba_oppotunity(home, away, df)
    if proba is not None:
        tmp = tmp.append(proba)
#    proba = proba_month(home, month, df)
#    if proba is not None:
#        tmp = tmp.append(proba)
    proba = proba_quarter(home, month, df)
    if proba is not None:
        tmp = tmp.append(proba)
    proba = proba_studium(home, studium, df)
    if proba is not None:
        tmp = tmp.append(proba)
    return tmp.product().idxmax(), tmp

In [14]:
result, pred = prediction("川崎Ｆ","札幌",9,"等々力",df)
print(result)
display(pred)

win


,win,draw,lose
all,0.510230,0.186701,0.303069
oppotunity,0.761905,0.190476,0.047619
quarter,0.384615,0.269231,0.346154
studium,0.593176,0.186352,0.220472


## アウェイチームから見た場合のデータも考える

ここまでだとホームチームから見た場合の確率しか考えていないので、アウェイチームから見た場合も考えてみる。  
確率の計算としては、単にHOME/AWAYを入れ替えるだけで同じような計算が可能。  
全く同じ方法で当てはめてみる。

In [15]:
result, pred = prediction("札幌","川崎Ｆ",9,"等々力",df)
print(result)
display(pred)

lose


,win,draw,lose
all,0.374554,0.219976,0.405470
oppotunity,0.047619,0.190476,0.761905
quarter,0.386364,0.193182,0.420455
studium,0.083333,0.083333,0.833333


## 結果の評価を行う

結果はホーム側からみた予想とアウェイ側から見た予想が双方出てくるので、下のような関係性で最終結果を出すものとする。

|アウェイ\ホーム|WIN|DRAW|LOSE|
|-------------|----|----|----|
|WIN          |DRAW|LOSE|LOSE|
|DRAW         |WIN |DRAW|LOSE|
|LOSE         |WIN |WIN |DRAW|

※最終結果はホームチームから見た結果

2016年以前のデータを使い、2017年のデータを予測して、精度を評価する。  
対象はJ1/J2とする。

In [16]:
data = df[df["year"] < 2017]
data = data.dropna()
answer = df[(df["year"] == 2017) & ((df["大会"]=="Ｊ１") | (df["大会"]=="Ｊ２"))].reset_index()
answer["result"] = answer["result"].replace(3,"win")
answer["result"] = answer["result"].replace(1,"draw")
answer["result"] = answer["result"].replace(0,"lose")
#display(answer)

In [17]:
from tqdm import tqdm_notebook
pred_home = []
pred_away = []
pred_result = []
for i in tqdm_notebook(range(len(answer))):
    tmp1, tmp2 = prediction(answer["ホーム"].iloc[i],answer["アウェイ"].iloc[i],answer["month"].iloc[i],answer["スタジアム"].iloc[i],data)
    tmp3, tmp4 = prediction(answer["アウェイ"].iloc[i],answer["ホーム"].iloc[i],answer["month"].iloc[i],answer["スタジアム"].iloc[i],data)
    pred_home.append(tmp1)
    pred_away.append(tmp3)
    if pred_home[i] == pred_away[i]:
        pred_result.append("draw")
    elif pred_home[i] == "win" or (pred_home[i] == "draw" and pred_away[i] == "lose"):
        pred_result.append("win")
    else:
        pred_result.append("lose")
        
answer["予想1"] = pred_home
answer["予想2"] = pred_away
answer["最終予想"] = pred_result
#display(answer)
print(len(answer[answer["result"]==answer["最終予想"]])/len(answer))


0.3815104166666667


結果、38%という数値になった。  
[ベイズ分類器](005_ベイズ分類器.ipynb)で試したBernoulli分布に即した方が結果良さそう？